<a href="https://colab.research.google.com/github/zwubbena/edfacts-metadata-deadline/blob/main/EDFacts_Metadata_Deadline_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDFacts Metadata Deadline Calculator for IDEA Data

This notebook calculates internal metadata submission deadlines for the following EDFacts File Specifications (FS) based on their federal due dates or deadlines.

## **File Specifications (FS)**
- **FS002** - Children with Disabilities (IDEA) School Age
- **FS089** - Children with Disabilities (IDEA) Early Childhood
- **FS009** - Children with Disabilities (IDEA) Exiting Special Education
- **FS005** - Children with Disabilities (IDEA) Removal to Interim - Alternative Educational Setting
- **FS006** - Children with Disabilities (IDEA) Suspensions/Expulsions
- **FS007** - Children with Disabilities (IDEA) Reasons for Unilateral Removal
- **FS088** - Children with Disabilities (IDEA) Disciplinary Removals
- **FS143** - Children with Disabilities (IDEA) Total Disciplinary Removals
- **FS144** - Educational Services During Expulsion

## **Rule**

### Internal Rule:
The internal rule ensures sufficient time for internal validation, error resolution, and coordination before the federal upload of EDFacts in EDPass. The internal rule for metadata submission is:
> **Internal Deadline** = One business day prior to 5 weeks before the federal EDFacts file specification (FS) deadline in EDPass

This means you:
1.	Count five weeks back from the federal submission date.
2.	Then go one business day earlier than that.

### Internal Rule Example:

For FS089, the federal deadline is Wednesday, July 30, 2025:

- Five weeks before = Wednesday, June 25, 2025
- One business day prior = Tuesday, June 24, 2025

So, the **internal metadata submission deadline would be Tuesday, January 13, 2026**.

## **How to Use**

1. Click the "Play button" to execute the Python script
2. Enter the federal due date (MM/DD/YYYY) for any FS row.
3. Click **"Calculate Deadlines"**.
4. View the results in the table below.
5. Download the generated PDF from the file panel on the left.


## **Output**

- A table is rendered within Colab and a downloadable PDF is generated. Both the table and the PDF include the FS names, federal deadlines, and internal metadata deadlines.


In [19]:
!pip install fpdf

from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from fpdf import FPDF

# Function to calculate internal metadata deadline
def calculate_internal_metadata_deadline(federal_due_date_str):
    try:
        federal_due_date = datetime.strptime(federal_due_date_str, "%m/%d/%Y")
        five_weeks_prior = federal_due_date - timedelta(weeks=5)
        one_day = np.busday_offset(five_weeks_prior.date(), -1, roll='backward')
        one_day = pd.to_datetime(one_day)
        return one_day.strftime("%m/%d/%Y")
    except:
        return "No date entered"

# FS labels
fs_labels = [
    "FS002 - Children with Disabilities (IDEA) School Age",
    "FS089 - Children with Disabilities (IDEA) Early Childhood",
    "FS009 - Children with Disabilities (IDEA) Exiting Special Education",
    "FS005 - Children with Disabilities (IDEA) Removal to Interim Alternative Educational Setting",
    "FS006 - Children with Disabilities (IDEA) Suspensions/Expulsions",
    "FS007 - Children with Disabilities (IDEA) Reasons for Unilateral Removal",
    "FS088 - Children with Disabilities (IDEA) Disciplinary Removals",
    "FS143 - Children with Disabilities (IDEA) Total Disciplinary Removals",
    "FS144 - Educational Services During Expulsion"
]

# Create input widgets
fs_widgets = []
for label in fs_labels:
    date_widget = widgets.Text(
        value='',
        placeholder='MM/DD/YYYY',
        layout=widgets.Layout(width='130px')
    )
    fs_widgets.append((label, date_widget))

# Button and output
submit_button = widgets.Button(description="Calculate Deadlines")
output = widgets.Output()

# Button click handler
def on_button_click(b):
    results = []
    for label, date_widget in fs_widgets:
        date_str = date_widget.value.strip()
        deadline = calculate_internal_metadata_deadline(date_str) if date_str else "No date entered"
        results.append({
            "File Specification": label,
            "Federal Due Date": date_str if date_str else "No date entered",
            "Internal Metadata Deadline": deadline
        })

    # Render table within Colab
    with output:
        clear_output()
        df = pd.DataFrame(results)
        styled_df = df.style.set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left')]},
            {'selector': 'td', 'props': [('text-align', 'left')]}
        ]).hide(axis='index')
        display(styled_df)

    # Generate PDF
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=11)
    pdf.cell(200, 10, txt="EDFacts Metadata Deadlines", ln=True, align='L')
    pdf.ln(5)
    for row in results:
        fs_text = row['File Specification']
        fed_date = row['Federal Due Date']
        internal_date = row['Internal Metadata Deadline']

        pdf.set_font("Arial", style='', size=11)
        pdf.multi_cell(0, 10, fs_text, align='L')

        pdf.set_font("Arial", style='B', size=11)
        pdf.cell(60, 10, "Federal Due Date:", ln=0)
        pdf.set_font("Arial", style='', size=11)
        pdf.cell(0, 10, fed_date, ln=1)

        pdf.set_font("Arial", style='B', size=11)
        pdf.cell(70, 10, "Internal Metadata Deadline:", ln=0)
        pdf.set_font("Arial", style='', size=11)
        pdf.cell(0, 10, internal_date, ln=1)

        pdf.ln(2)

    pdf.output("metadata_deadlines.pdf")
    with output:
        print("📄 PDF generated: metadata_deadlines.pdf (check the file panel on the left to download)")

submit_button.on_click(on_button_click)

# Display layout
rows = [widgets.HBox([
    widgets.Label(value=label, layout=widgets.Layout(width='65%')),
    date_widget
]) for label, date_widget in fs_widgets]

display(widgets.VBox(rows + [submit_button, output]))
